In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

Executing: /tmp/apt-key-gpghome.9fDijlHEK5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://downloads.skewed.de/apt jammy InRelease [7,535 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://downloads.skewed.de/apt jammy/main amd64 Packages [3,167 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/

In [4]:
# Colab uses a Python install that deviates from the system's! Bad collab! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be REMOVED:
  python3-cairo*
0 upgraded, 0 newly installed, 1 to remove and 40 not upgraded.
After this operation, 310 kB disk space will be freed.
(Reading database ... 130121 files and directories currently installed.)
Removing python3-cairo:amd64 (1.20.1-3build1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libffi-dev libglib2.0-dev libglib2.0-dev-bin libice-dev
  liblzo2-2 libmount-dev libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev libxcb-render0-dev
  libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libg

In [5]:
import graph_tool.all as gt
import networkx as nx
import random
import itertools as it
import numpy as np


g = gt.collection.ns["us_air_traffic"]
print('done')

done


In [9]:
# Get node properties
node_properties = {key: g.vertex_properties[key] for key in g.vertex_properties.keys()}

# Get edge properties
edge_properties = {key: g.edge_properties[key] for key in g.edge_properties.keys()}


# Define the list of node properties to remove
properties_to_remove = ["airport_code","airport_id","airport_seq_id","city_market_id" ,"origin"
 ,"state_fips","state_nm","wac"]

# Remove node properties
for prop_name in properties_to_remove:
    if prop_name in g.vertex_properties:
        del g.vertex_properties[prop_name]

for prop_name in list(g.edge_properties.keys()):
    del g.edge_properties[prop_name]


print("Node properties:")
for key, prop in node_properties.items():
    print(key, prop)

print("\nEdge properties:")
for key, prop in edge_properties.items():
    print(key, prop)


Node properties:
city_name <VertexPropertyMap object with value type 'string', for Graph 0x78ead81c80d0, at 0x78ead7c84940>
state_abr <VertexPropertyMap object with value type 'string', for Graph 0x78ead81c80d0, at 0x78ead7c84b20>
_pos <VertexPropertyMap object with value type 'vector<double>', for Graph 0x78ead81c80d0, at 0x78ead7c84ac0>

Edge properties:


In [7]:
g.save("network.gml")

In [8]:
# Get Network Stats
g = nx.read_gml("network.gml")


NetworkXError: input is not ASCII-encoded

In [10]:
# Bidirectional Percentage

num_bidirectional_edges = 0
for e in g.edges():
    src, tgt = int(e.source()), int(e.target())
    if g.edge(tgt, src):
        num_bidirectional_edges += 1

# Calculate the total number of edges
num_edges = g.num_edges()

# Calculate the percentage of bidirectional edges
bidirectional_percentage = (num_bidirectional_edges / num_edges) * 100

In [11]:
# Degree analysis
# Calculate the degree of each vertex
degrees = [v.out_degree() for v in g.vertices()]

# Calculate average, min, and max degree
avg_degree = sum(degrees) / len(degrees)
min_degree = min(degrees)
max_degree = max(degrees)


# Calculate the average clustering coefficient

#assortativity_coefficient = gt.assortativity(g, "total")


In [13]:
import json
from datetime import date

# Gather information about the graph's structure
structure_info = {
    "Number of nodes": g.num_vertices(),
    "Number of links": g.num_edges(),
    "--- Bidirectional links": bidirectional_percentage, " %" # Calculate bidirectional links percentage
    "Degree ": [str(avg_degree), "[ "+ str(min_degree) +" , " + str(max_degree) + " ]", "Distributions summarized with average [min, max]"],
    "Connected": "Yes",  # Determine connectivity

}

# Gather information about the graph's metadata
metainfo = {
    "Node metadata": "City Name, State Abbr. and position (lat/long)",
    "Link metadata": "None",
    "Date of creation": str(date.today()),
    "Data generating process": "Network data was downloaded from the link below and the data was process via the graph-tools package",
    "Ethics": "n/a",
    "Funding": "None",
    "Citation": "None",
    "Access": "https://networks.skewed.de/net/us_air_traffic#None_draw"
}

# Construct the JSON object
network_card = {
    "overall": {
        "Name": "US Flights",
        "Kind": "Directed, unweighted",
        "Nodes are": "Airports",
        "Links are": "Flights between airports",
    },
    "structure": structure_info,
    "metainfo": metainfo
}

# Save the JSON object to a file
filename = "graph_network_card.json"
with open(filename, "w") as json_file:
    json.dump(network_card, json_file, indent=4)

print(f"Network card saved to {filename}")

Network card saved to graph_network_card.json
